# Inleiding
In de beschrijving van de theorie vind je een aantal losse oefeningen. We zullen deze oefeningen aanvullen door één geïntegreerde oefening waar de belangrijkste vaardigheden van dit hoofdstuk aan bod komen.
We zullen in deze oefening de volgende stappen zetten:
* We lezen gegevens uit de sqlite3-database Chinook. Die gegevens komen terecht in een dataframe.
* We maken een aantal subframes waarin we data filteren.
* We berekenen een aantal statistische gegevens uit ons dataframe.
* We schrijven de diverse resultaten weg naar een excelbestand met diverse tabbladen.

# De database
We gebruiken in deze oefening de sqlite3-database chinook. Sqlite-bestanden bevatten een relationele database. De specifiek eigenschappen van sqlite zijn:

* Het vraagt geen serverinfrastructuur. De database is opgeslagen in één bestand, er moeten geen server opgezet worden of een configuratie gemaakt worden. Daarom vraagt sqlite weinig systeembronnen. Dat maakt het zo geschikt voor gebruik in IoT-devices, maar ook als gegevensopslag voor de lokale verwerking van grote hoeveelheden complexe gegevens.
* Sqlite3 is beschikbaar op alle platformen (Windows, Mac, Linux, Android, FreeBSD, ...).
* De python-bibliotheek om sqlite-bestanden te lezen is meegeleverd bij de installatie van Python. Je hoeft niets extra te installeren.
* Ondanks zijn eenvoudig opzet is sqlite een volwaardige relationele database, met de mogelijkheid voor transacties volgens het ACID-principe (atomic, consistent, isolated, durable).
* Sqlite kan terrabytes aan gegevens verwerken. Dat maakt het een uitstekend alternatief voor Excelbestanden of tekstbestanden.

De database die we zullen gebruiken is de Chinook-database, een gratis demo die je op het web kunt vinden. Meer informatie over de sqlite-database Chinook vind je onder andere [hier](https://www.sqlitetutorial.net/sqlite-sample-database/). Je kan hem voor diverse DBMS'en ook [hier](https://github.com/lerocha/chinook-database) vinden.

Je kan heel gemakkelijk de chinook-database verkennen in het programma [DB Browser for Sqlite](https://sqlitebrowser.org/). Dit is een gebruiksvriendelijke programma voor diverse platformen waarmee je gemakkelijk gegevens uit sqlite-databanken kan raadplegen en wijzigen, via een gebruiksvriendelijke GUI of via SQL. Het is een ideale manier om je SQL-commando uit te testen.

# De gegevens uit de database halen
De eerste stap is de gegevens uit de database halen en in een pandas-dataframe stoppen.
Schrijf de Pythoncode om de volgende gegevens uit de database te halen:

* Van elke track geef je de factuurlijnen, met de volgende kolommen: naam van de Track, naam van het album waarin die track voorkwam, naam van de artis van dat album, naam van het genre van de track, verkochte hoeveelheid, prijs waartegen de track verkocht werd.
* Geef de kolommen de volgende kolomnamen: Name, Artist, Album, Genre, UnitPrice, Quantity.
* Je zal hiervoor vier tabellen moeten combineren met een join.
* De tracks waarvan geen factuurregel is (die dus ook nooit verkocht werden), moeten ook in de lijst voorkomen. Je zal dus een outer join moeten doen.

Sla die gegevens op in de dataframe `tracks`. Je zou 3759 rijen moeten hebben.

In [1]:
import sqlite3
from sqlite3 import Error

try:
  conn = sqlite3.connect("chinook.db")
except Error as e:
  print(e)

import pandas as pd
sql_query = '''
select t.Name, alb.Title as Album, art.Name as Artist, g.Name as Genre, inv.UnitPrice as UnitPrice, inv.Quantity as Quantity
from tracks t left outer join invoice_items inv on inv.TrackId = t.TrackId
join genres g on t.GenreId = g.GenreId
join albums alb on t.AlbumId = alb.Albumid
join artists art on alb.ArtistId = art.ArtistId
order by t.Name;
'''
tracks = pd.read_sql_query(sql_query, conn)
print(tracks.count())

Name         3759
Album        3759
Artist       3759
Genre        3759
UnitPrice    2240
Quantity     2240
dtype: int64


# Gegevens filteren
We gaan nu wat oefenen met het filteren van gegevens. Toon achtereenvolgens de volgende gegevens:

* Geef de eerste 10 tracks uit de lijst.
* Geef de tracks die nooit verkocht werden. Geef de naam van het track en de artist (Aantal is 1519). Sla die op een een dataframe `unsold`.
* Geef de tracks van het genre Rock en van het genre Blues (Aantal is 1476).
* Geef de tracks van Gilberto Gil die ook van het genre Soundtrack zijn (Aantal is 14).
* Geef de tracks van Iron Maiden die niet onder het genre Heavy Metal vallen (Aantal is 202).

In [2]:
# Geef de eerste 10 tracks uit de lijst.
tracks.head(10)

,Name,Album,Artist,Genre,UnitPrice,Quantity
0,"""40""",War,U2,Rock,NaN,NaN
1,"""?""","Lost, Season 2",Lost,TV Shows,1.99,1.0
2,"""Eine Kleine Nachtmusik"" Serenade In G, K. 525...",Sir Neville Marriner: A Celebration,Academy of St. Martin in the Fields Chamber En...,Classical,NaN,NaN
3,#1 Zero,Out Of Exile,Audioslave,Alternative & Punk,NaN,NaN
4,#9 Dream,Instant Karma: The Amnesty International Campa...,U2,Pop,0.99,1.0
5,'Round Midnight,The Essential Miles Davis [Disc 1],Miles Davis,Jazz,0.99,1.0
6,(Anesthesia) Pulling Teeth,Kill 'Em All,Metallica,Metal,0.99,1.0
7,(Da Le) Yaleo,Supernatural,Santana,Rock,NaN,NaN
8,(I Can't Help) Falling In Love With You,UB40 The Best Of - Volume Two [UK],UB40,Reggae,NaN,NaN
9,(Oh) Pretty Woman,Diver Down,Van Halen,Rock,NaN,NaN


In [3]:
# Geef de tracks die nooit verkocht werden. Geef de naam van het track en de artist (Aantal is 1519).
unsold = tracks.loc[tracks['Quantity'].isna(), ["Name", "Artist"]]
unsold

,Name,Artist
0,"""40""",U2
2,"""Eine Kleine Nachtmusik"" Serenade In G, K. 525...",Academy of St. Martin in the Fields Chamber En...
3,#1 Zero,Audioslave
7,(Da Le) Yaleo,Santana
8,(I Can't Help) Falling In Love With You,UB40
...,...,...
3752,É Preciso Saber Viver,Titãs
3753,É Uma Partida De Futebol,Skank
3754,É que Nessa Encarnação Eu Nasci Manga,Various Artists
3757,Óia Eu Aqui De Novo,Gilberto Gil


In [4]:
# Geef de tracks van het genre Rock en van het genre Blues (Aantal is 1476).
tracks.loc[(tracks["Genre"] == 'Blues') | (tracks["Genre"] == 'Rock'),:]

,Name,Album,Artist,Genre,UnitPrice,Quantity
0,"""40""",War,U2,Rock,NaN,NaN
7,(Da Le) Yaleo,Supernatural,Santana,Rock,NaN,NaN
9,(Oh) Pretty Woman,Diver Down,Van Halen,Rock,NaN,NaN
13,(Wish I Could) Hideaway,"Chronicle, Vol. 2",Creedence Clearwater Revival,Rock,0.99,1.0
18,01 - Prowler,Iron Maiden,Iron Maiden,Blues,0.99,1.0
...,...,...,...,...,...,...
3740,Zooropa,Zooropa,U2,Rock,0.99,1.0
3741,Zé Trindade,O Samba Poconé,Skank,Rock,0.99,1.0
3746,Às Vezes,Compositores,O Terço,Rock,0.99,1.0
3747,Água E Fogo,Maquinarama,Skank,Rock,0.99,1.0


In [5]:
# Geef de tracks van Gilberto Gil die ook van het genre Soundtrack zijn (Aantal is 14).
tracks.loc[(tracks["Artist"] == 'Gilberto Gil') & (tracks["Genre"] == 'Soundtrack'),:]

,Name,Album,Artist,Genre,UnitPrice,Quantity
96,A Volta Da Asa Branca,As Canções de Eu Tu Eles,Gilberto Gil,Soundtrack,NaN,NaN
232,As Pegadas Do Amor,As Canções de Eu Tu Eles,Gilberto Gil,Soundtrack,NaN,NaN
239,Asa Branca,As Canções de Eu Tu Eles,Gilberto Gil,Soundtrack,NaN,NaN
245,Assum Preto,As Canções de Eu Tu Eles,Gilberto Gil,Soundtrack,0.99,1.0
289,Baião Da Penha,As Canções de Eu Tu Eles,Gilberto Gil,Soundtrack,NaN,NaN
567,Casinha Feliz,As Canções de Eu Tu Eles,Gilberto Gil,Soundtrack,NaN,NaN
974,Esperando Na Janela,As Canções de Eu Tu Eles,Gilberto Gil,Soundtrack,0.99,1.0
1590,Juazeiro,As Canções de Eu Tu Eles,Gilberto Gil,Soundtrack,0.99,1.0
1656,Lamento Sertanejo,As Canções de Eu Tu Eles,Gilberto Gil,Soundtrack,0.99,1.0
2161,O Amor Daqui De Casa,As Canções de Eu Tu Eles,Gilberto Gil,Soundtrack,NaN,NaN


In [6]:
# Geef de tracks van Iron Maiden die niet onder het genre Heavy Metal vallen (Aantal is 202).
tracks.loc[(tracks["Artist"] == 'Iron Maiden') & (tracks["Genre"] != 'Heavy Metal')]

,Name,Album,Artist,Genre,UnitPrice,Quantity
18,01 - Prowler,Iron Maiden,Iron Maiden,Blues,0.99,1.0
19,02 - Sanctuary,Iron Maiden,Iron Maiden,Blues,NaN,NaN
20,03 - Remember Tomorrow,Iron Maiden,Iron Maiden,Blues,NaN,NaN
21,04 - Running Free,Iron Maiden,Iron Maiden,Blues,0.99,1.0
22,05 - Phantom of the Opera,Iron Maiden,Iron Maiden,Blues,0.99,1.0
...,...,...,...,...,...,...
3617,Where Eagles Dare,A Real Dead One,Iron Maiden,Metal,0.99,1.0
3618,Where Eagles Dare,Piece Of Mind,Iron Maiden,Metal,0.99,1.0
3619,Where Eagles Dare,Piece Of Mind,Iron Maiden,Metal,0.99,1.0
3680,Wrathchild,Live At Donington 1992 (Disc 1),Iron Maiden,Rock,0.99,1.0


# Een totaalkolom toevoegen
Voeg een extra kolom toe met het verkochte bedrag per factuurregel (bedrag * aantal). Geef die kolom de naam Total.
Wijzig de totaalkolommen zonder waarde. Zet daar de waarde op 0.

In [7]:
tracks["Total"] = tracks["UnitPrice"] * tracks["Quantity"]
tracks.loc[tracks["Total"].isna(),["Total"]] = 0
tracks

,Name,Album,Artist,Genre,UnitPrice,Quantity,Total
0,"""40""",War,U2,Rock,NaN,NaN,0.00
1,"""?""","Lost, Season 2",Lost,TV Shows,1.99,1.0,1.99
2,"""Eine Kleine Nachtmusik"" Serenade In G, K. 525...",Sir Neville Marriner: A Celebration,Academy of St. Martin in the Fields Chamber En...,Classical,NaN,NaN,0.00
3,#1 Zero,Out Of Exile,Audioslave,Alternative & Punk,NaN,NaN,0.00
4,#9 Dream,Instant Karma: The Amnesty International Campa...,U2,Pop,0.99,1.0,0.99
...,...,...,...,...,...,...,...
3754,É que Nessa Encarnação Eu Nasci Manga,Axé Bahia 2001,Various Artists,Pop,NaN,NaN,0.00
3755,"Étude 1, In C Major - Preludio (Presto) - Liszt",Liszt - 12 Études D'Execution Transcendante,Michele Campanella,Classical,0.99,1.0,0.99
3756,Óculos,Arquivo Os Paralamas Do Sucesso,Os Paralamas Do Sucesso,Latin,0.99,1.0,0.99
3757,Óia Eu Aqui De Novo,As Canções de Eu Tu Eles,Gilberto Gil,Soundtrack,NaN,NaN,0.00


# Statistische gegevens
* Maak een nieuw dataframe met de lijst van de verkochte bedragen per Genre. Geef het de naam verkocht_per_genre.
* Maak een nieuw dataframe met de lijst van de verkochte bedragen per artist. Geef het de naam verkocht_per_artist.
* Wat is de artist die het meest verkocht heeft?

In [8]:
# Maak een nieuw dataframe met de lijst van de verkochte bedragen per Genre.
verkocht_per_genre = tracks[["Genre","Total"]].groupby("Genre").mean()
verkocht_per_genre

,Total
Genre,
Alternative,0.346500
Alternative & Punk,0.647614
Blues,0.678539
Bossa Nova,0.928125
Classical,0.513797
Comedy,0.995000
Drama,0.874394
Easy Listening,0.412500
Electronica/Dance,0.383226


In [9]:
# Maak een nieuw dataframe met de lijst van de verkochte bedragen per artist. 
verkocht_per_artist = tracks[["Artist", "Total"]].groupby("Artist").sum()
verkocht_per_artist

,Total
Artist,
AC/DC,15.84
Aaron Copland & London Symphony Orchestra,0.00
Aaron Goldberg,0.00
Academy of St. Martin in the Fields & Sir Neville Marriner,0.99
Academy of St. Martin in the Fields Chamber Ensemble & Sir Neville Marriner,0.00
...,...
Vinícius De Moraes,5.94
Wilhelm Kempff,0.00
Yehudi Menuhin,0.99


In [10]:
# best verkochte artist
verkocht_per_artist.sort_values(by="Total", ascending=False).head(1)

,Total
Artist,
Iron Maiden,138.6


# Wegschrijven naar excel
Maak een Excelbestand met de volgende tabbladen:

* Alle factuurlijnen.
* Alle tracks die nooit verkocht werden.
* De lijst van de verkochte bedragen per genre.
* De lijst van de verkochte bedragen per artist.

In [11]:
import openpyxl
with pd.ExcelWriter('chinook-statistieken.xlsx') as writer:
    tracks.to_excel(writer, 'Alle factuurlijnen')
    unsold.to_excel(writer, 'Niet-verkochte tracks')
    verkocht_per_genre.to_excel(writer, 'Per Genre')
    verkocht_per_artist.to_excel(writer, 'Per Artist')